In [2]:
from datascience import *
from collections import Counter
import numpy as np
%matplotlib inline

# Old Bailey Classification: Problemset 3

### Question 2: Old Bailey Data

#### 2.1: Outcome Category

outcome: outcomes for defendant
feature extraction and add column for true false before this policy change 

#### 2.2: Time Period

According to the Old Bailey website, Until 1858 jury would be kept without fire, food or drink until a verdict was agreed. Their decisions normally took very little time to determine. Hence, I am interested in exploring the differences in the in these outcomes in the 20 years prior and after this change in policy.

#### 2.3: The Data

In [3]:
import xml.etree.cElementTree as ET  # XML parser
import requests  # make requests to web servers
import time  # will help us pause python's for loop
import json

In [4]:
trials = requests.get("https://www.oldbaileyonline.org/obapi/ob?term0=fromdate_18380101&term1=todate_18781209&&start=0").json()

In [5]:
trials

{'hits': ['t18380101-312',
  't18380101-313',
  't18380101-314',
  't18380101-315',
  't18380101-316',
  't18380101-317',
  't18380101-318',
  't18380101-319',
  't18380101-320',
  't18380101-321',
  't18380101-322',
  't18380101-323',
  't18380101-324',
  't18380101-325',
  't18380101-326',
  't18380101-327',
  't18380101-328',
  't18380101-329',
  't18380101-330',
  't18380101-331',
  't18380101-332',
  't18380101-333',
  't18380101-334',
  't18380101-335',
  't18380101-336',
  't18380101-337',
  't18380101-338',
  't18380101-339',
  't18380101-340',
  't18380101-341',
  't18380101-342',
  't18380101-343',
  't18380101-344',
  't18380101-345',
  't18380101-346',
  't18380101-347',
  't18380101-348',
  't18380101-349',
  't18380101-350',
  't18380101-351',
  't18380101-352',
  't18380101-353',
  't18380101-354',
  't18380101-355',
  't18380101-356',
  't18380101-357',
  't18380101-358',
  't18380101-359',
  't18380101-360',
  't18380101-361',
  't18380101-362',
  't18380101-363',
  't

In [24]:
# iterate through manuscripts
for trial in trials['hits'][:100]:

    # build url
    url = 'http://www.oldbaileyonline.org/obapi/text?div={}'.format(trial)
        
    # get the response
    res = requests.get(url).text
        
    #create a file name
    fname = 'data/old-bailey-'+ '-' + trial + '.xml'
    
    # save the file
    with open(fname, 'w') as f:
            f.write(res)

    # pause for a second so we don't overload their servers
    time.sleep(1)



### Question 3: Exploratory Data Analysis/Data Preperation

In [6]:
import pandas as pd
import xml.etree.cElementTree as ET #XML Parser
from lxml import etree #ElementTree and lxml allow us to parse the XML file.
import requests #make request to server
import time #pause loop

#### 3.1: Preprocess the Text
Use techniques covered in labs such as tokenizing, stemming, and chunking, as well as any other methods you might find helpful. In a few sentences, explain your choice of techniques, and your general goals at this stage.


In [29]:
def collect_text(element):
    iterate = file.getiterator()
    for element in iterate:
        text_list = []
        if element.tag == 'p':
            text = ''.join(list(element.itertext()))
            text_list.append(text)
    return text_list   

In [27]:
def table_of_cases(xml_file_name):
    file = ET.ElementTree(file = xml_file_name)
    iterate = file.getiterator()
    i = 1
    table = pd.DataFrame()
    for element in iterate:
        if element.tag == "interp":
            t = element.attrib['type']
            val = [element.attrib['value']]
            labels = list(table.columns.values)
            num = str(i)
            if t not in labels:
                table[t] = val
            elif t+num not in labels:
                table[t+num] = val
            elif t+num in labels:
                num = str(i+1)
                table[t+num] = val
    return table

In [28]:
df = pd.DataFrame()
for i in trials['hits'][:100]:
    raw_data = 'data/old-bailey--'+ i +'.xml'
    data = table_of_cases(raw_data)
    df = df.append(data, ignore_index=True)
df

ValueError: Length of values does not match length of index

In [14]:
for (index, value) in df['text'].iteritems(): 
    print(value)
#df['text']


























Before Mr. Justice Coltman.
Before Mr. Baron Alderson.
Before Mr. Justice Williams.
Before Mr. Justice Coltman.
Before Mr. Baron Alderson.
Before Mr. Justice Williams.
Before Mr. Baron Alderson.
Third Jury, before Mr. Recorder.






Recommended to mercy by the Jury.
(There was another indictment against the prisoner.)

















(There was another indictment against the prisoner.)
Fifth Jury, before Mr. Sergeant Arabin.













Before Mr. Baron Alderson.
Fourth Jury, before Mr. Recorder.


(There were two other indictments against the prisoner.)















Sixth Jury, before Mr. Sergeant Arabin.




(There was another indictment against the prisoner.)



#### 3.2: Exploratory Data Findings

You might report word frequencies, clustering patterns, etc. that might help with your analysis. Feel free to get creative here!

### Question 4: Build a Classification Algorithm

#### 4.1: Feature extraction 

In class, we used simple word frequencies and tf-idf. You might use these, extend the tf-idf with more specifications, or another method entirely. Explain the underlying assumptions for your chosen method, and explain its advantages and disadvantages.

#### 4.2: Machine Learning Techniques

Choose 3 different machine learning/prediction techniques. Detail the basic logic underlying each, the pros and cons, whether it is parametric or non- parametric etc. Also discuss why it may be an appropriate choice for this particular problem.
In class we learned Logistic Regression, Naive Bayes, and Support Vector Machine techniques. You are strongly encouraged to explore other methods as well.

#### 4.3: Training, Validation, and Test Sets Split

Report the classification accuracy in each validation set. Choose your best model, and report its accuracy on the test set.

### Question 5: Dictionary Methods

#### Question 5.1: "Confirmed", "suspect", and "ignore" Categories
Confirmed words are ones that you are confident will aid with classification, suspect words are those that you are unsure of, and ignore words are ones that should not be used for classification. Please note that you should use a manageable subset of words for this task - no more than 100 words or so.


#### 5.2: Words Associated with "Suspect"

Create a table matching suspect words with their associated words, and decide whether to include them in the algoirthm. Choose some examples to explain your reasoning

#### 5.3: Form a Dictionary
Choose your favorite classification algorithm, and train/test the model. Report your accuracy.


### Question 6: Discussion Questions

#### 6.1: How did the dictionary method’s accuracy compare to the alorithm trained on the full set of features?
What are the advantages and disadvantages of employing each approach?

#### 6.2: Do you think these rules would generalize well to a different set of docu- ments, say U.S. courts today? Why or why not?


#### 6.3 Discuss any interesting findings. Restate your initial research question, and time period. Evaluate your initial hypothesis, and discuss any conclusions you can draw from your analysis. Identify future areas for research.